# IBM Capstone

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
import requests
from bs4 import BeautifulSoup

## Get data from wikipedia

In [3]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text


soup = BeautifulSoup(url,"lxml")
#print(soup.prettify())




table = soup.find("table",{"class":"wikitable sortable"})


parsed_table_data = []



rows = table.findAll('tr')
for row in rows:
                chicken = row.findChildren(recursive=False)
                row_text = []
            
                for chick in chicken:
                    clean_text = chick.text.strip()
                    row_text.append(clean_text)
  
            
                parsed_table_data.append(row_text)
                
                


df=pd.DataFrame(parsed_table_data,columns=["Postcode","Borough","Neighbour"])


df.head()

,Postcode,Borough,Neighbour
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


## Data cleaning

In [17]:
df=df[df.Borough!="Not assigned"]
df=df[df.Postcode!="Postcode"]
df.shape
df.loc[df.Neighbour=="Not assigned","Neighbour"] = df.Borough
df.head()

AttributeError: 'DataFrame' object has no attribute 'Postcode'

## Group by postcode

In [ ]:
df1 = (df.groupby(['Postcode','Borough'])['Neighbour']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())
df1.columns=["PostCode","Borough","Neighbour"]
df1.head()

In [18]:
df=df1
df.shape

(103, 3)

## GEO Data

In [19]:
geo_df = pd.read_csv("http://cocl.us/Geospatial_data")
geo_df.columns=["PostCode","Latitude","Longitude"]
geo_df.head()

,PostCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df=pd.merge(df,geo_df,on="PostCode")
df.head()

,PostCode,Borough,Neighbour,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Filtering rows with the value Toronto

In [21]:
df=df.loc[df['Borough'].str.contains("Toronto")]
df.head()

,PostCode,Borough,Neighbour,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Nearby venues

In [23]:
df.shape

(38, 5)

## Position of Toronto

In [24]:
address = 'Toronto,CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [25]:
df.head()

,PostCode,Borough,Neighbour,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [26]:
neighborhoods=df
neighborhoods.head()

,PostCode,Borough,Neighbour,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Let's check the size of the resulting dataframe

## A map of Toronto with neighborhoods superimposed

In [27]:
map_t = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbour']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_t)  
    
map_t